In [57]:
import pandas as pd
import numpy as np

In [74]:
# pull down data from the Expense Tagging repo
df = pd.read_csv("https://github.com/EZ-Walk/Expense-Tagging/blob/f6b58fbcc1270d6bcdab82873673ee34c6d1e84a/Data/raw/AccountHistory%20(1).csv?raw=true")
print(df.shape)
df.head()

(3509, 8)


,Account Number,Post Date,Check,Description,Debit,Credit,Status,Balance
0,443091309,4/27/2024,NaN,"GITHUB, INC. +18774484820 CAUS",10.00,NaN,Pending,NaN
1,443091309,4/25/2024,NaN,LIME*RIDE +18885463345 CAUS,4.44,NaN,Pending,NaN
2,443091309,4/27/2024,NaN,Deposit weekly allowance,NaN,120.0,Posted,116.43
3,443091309,4/27/2024,NaN,Point Of Sale Withdrawal MASABI_RTD 1600 Blake...,2.70,NaN,Posted,-3.57
4,443091309,4/27/2024,NaN,Point Of Sale Withdrawal SQ *CURTIS PARK 2532 ...,19.24,NaN,Posted,-0.87


In [75]:
# keep only rows with Debit!=NaN, Status=Posted, and Account Number == 4430913
df = df[df['Debit'].notnull()]
df = df[df['Status'] == 'Posted']
df = df[df['Account Number'] == 443091309]
print('Data shape:', df.shape)
df.head()

Data shape: (2726, 8)


,Account Number,Post Date,Check,Description,Debit,Credit,Status,Balance
3,443091309,4/27/2024,NaN,Point Of Sale Withdrawal MASABI_RTD 1600 Blake...,2.70,NaN,Posted,-3.57
4,443091309,4/27/2024,NaN,Point Of Sale Withdrawal SQ *CURTIS PARK 2532 ...,19.24,NaN,Posted,-0.87
5,443091309,4/27/2024,NaN,Point Of Sale Withdrawal SQ *CURTIS PARK 2532 ...,2.75,NaN,Posted,18.37
6,443091309,4/26/2024,NaN,Point Of Sale Withdrawal TST* FAMOUS ORI 713 E...,10.80,NaN,Posted,21.12
7,443091309,4/26/2024,NaN,External Withdrawal PAYPAL INSTANT TRANSFER - ...,60.00,NaN,Posted,31.92


In [82]:
# keep only the desired columns
data = df[['Post Date', 'Debit', 'Description']]
print('Data shape:', data.shape)
data.head()

Data shape: (2726, 3)


,Post Date,Debit,Description
3,4/27/2024,2.70,Point Of Sale Withdrawal MASABI_RTD 1600 Blake...
4,4/27/2024,19.24,Point Of Sale Withdrawal SQ *CURTIS PARK 2532 ...
5,4/27/2024,2.75,Point Of Sale Withdrawal SQ *CURTIS PARK 2532 ...
6,4/26/2024,10.80,Point Of Sale Withdrawal TST* FAMOUS ORI 713 E...
7,4/26/2024,60.00,External Withdrawal PAYPAL INSTANT TRANSFER - ...


In [83]:
# convert the 'Post Date' column to datetime
data['Date'] = data['Post Date'].astype('datetime64')
data

/var/folders/kd/x3brfgln0ng5q695bs9wjnkc0000gn/T/ipykernel_34054/1454478724.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Date'] = data['Post Date'].astype('datetime64')


,Post Date,Debit,Description,Date
3,4/27/2024,2.70,Point Of Sale Withdrawal MASABI_RTD 1600 Blake...,2024-04-27
4,4/27/2024,19.24,Point Of Sale Withdrawal SQ *CURTIS PARK 2532 ...,2024-04-27
5,4/27/2024,2.75,Point Of Sale Withdrawal SQ *CURTIS PARK 2532 ...,2024-04-27
6,4/26/2024,10.80,Point Of Sale Withdrawal TST* FAMOUS ORI 713 E...,2024-04-26
7,4/26/2024,60.00,External Withdrawal PAYPAL INSTANT TRANSFER - ...,2024-04-26
...,...,...,...,...
3502,5/24/2018,7.35,Point Of Sale Withdrawal DEBIT PURCHASE CHIC...,2018-05-24
3503,5/22/2018,45.25,Point Of Sale Withdrawal POS PURCHASE CORNER...,2018-05-22
3504,5/17/2018,45.66,External Withdrawal ANYTIME FIT ABC/CLUB FEES/...,2018-05-17
3505,5/8/2018,9.47,Point Of Sale Withdrawal DEBIT PURCHASE CHIP...,2018-05-08


In [ ]:
# 

In [79]:
# fill in dates without expenses as 0
data = data.set_index('Date').resample('D').sum().fillna(0)

KeyError: "None of ['Date'] are in the columns"

,Date,Debit
0,2018-05-07,10.00
1,2018-05-08,9.47
2,2018-05-09,0.00
3,2018-05-10,0.00
4,2018-05-11,0.00
...,...,...
2178,2024-04-23,623.02
2179,2024-04-24,0.00
2180,2024-04-25,26.84
2181,2024-04-26,208.10


In [70]:
# Print the shape one last time and the date range encomapssed by the data as a Month, Day, Year
print('Data shape:', data.shape)
date_range = data.index[[0, -1]]
print('Date range:', date_range[0].strftime('%B %d, %Y'), 'to', date_range[1].strftime('%B %d, %Y'))

Data shape: (2183, 1)
Date range: May 07, 2018 to April 27, 2024


In [37]:
data.to_csv('data/transactions.csv', index=False)


# Pre-processing

In [85]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import numpy as np

In [92]:
df = pd.read_csv('data/transactions.csv')
print(df.shape)
df.head()

(2183, 1)


,Debit
0,10.00
1,9.47
2,0.00
3,0.00
4,0.00


In [93]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
df['norm_Debit'] = scaler.fit_transform(df['Debit'].values.reshape(-1,1))
df.head()

,Debit,norm_Debit
0,10.00,0.000294
1,9.47,0.000278
2,0.00,0.000000
3,0.00,0.000000
4,0.00,0.000000


In [94]:
# Transform the data into sequences
sequence_length = 379
result = []
for index in range(len(df) - sequence_length):
    result.append(df['norm_Debit'].values[index: index + sequence_length])

result = np.array(result)

In [95]:
# Split the data into training and testing sets
train_size = round(0.9 * result.shape[0])  # 90% for training
x_train = result[:int(train_size), :-1]
y_train = result[:int(train_size), -1]
x_test = result[int(train_size):, :-1]
y_test = result[int(train_size):, -1]
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(1624, 378) (1624,) (180, 378) (180,)


In [96]:
# Save the training data
np.save('data/x_train.npy', x_train)
np.save('data/y_train.npy', y_train)

# Save the testing data
np.save('data/x_test.npy', x_test)
np.save('data/y_test.npy', y_test)